## Preparing Dataset
    * Checking the audio files and thier corresponding text files exists or not
    * Creating a csv with the existing pairs
    * Train and test splitting (using 20% for test)
    * Creating csv files for train and test

In [1]:
import os

In [3]:
!pwd

/home/ubuntu/Jairam/whisper-ASR


In [1]:
from huggingface_hub import login
use_auth_token = ''  # create and add the token from hugging face
login(use_auth_token)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/ubuntu/.cache/huggingface/token
Login successful


In [2]:
import os
import pandas as pd
import chardet

# Define the root directory
root_directory = "/Training"

# Initialize empty lists
file_paths = []
transcripts = []

# Iterate through the files in the audio_folder
audio_folder = os.path.join(root_directory, "Audio")
transcript_folder = os.path.join(root_directory, "Transcripts")

for audio_filename in os.listdir(audio_folder):
    audio_file_path = os.path.join(audio_folder, audio_filename)
    transcript_filename = audio_filename.replace(".wav", ".txt")
    transcript_file_path = os.path.join(transcript_folder, transcript_filename)

    # Debug prints to check the paths and filenames
    print(f"Audio file path: {audio_file_path}")
    print(f"Transcript file path: {transcript_file_path}")

    # Check if the corresponding transcript file exists
    if os.path.exists(transcript_file_path):
        # Check if the audio file and transcript file names match
        if transcript_filename == os.path.basename(transcript_file_path):
            file_paths.append(audio_file_path)
            try:
                with open(transcript_file_path, 'rb') as file:
                  rawdata = file.read()
                  result = chardet.detect(rawdata)
                  encoding = result['encoding']
                  print(f"Detected encoding: {encoding}")
                with open(transcript_file_path, 'r', encoding=encoding) as transcript_file:
                    transcript = transcript_file.read()
                transcripts.append(transcript)
            except UnicodeDecodeError:
                print(f"Error decoding file: {transcript_filename}")

        else:
            print(f"File names do not match for {audio_filename}")
    else:
        print(f"Transcript not found for {audio_filename}")

# Create a DataFrame if there are audio-transcript pairs
if len(file_paths) > 0:
    data = {'path': file_paths, 'transcript': transcripts}
    df = pd.DataFrame(data)

    # Display the DataFrame
    print(df)
else:
    print("No audio-transcript pairs found.")


Audio file path: /home/ubuntu/Jairam/Training/Audio/Audio - 27_01.wav
Transcript file path: /home/ubuntu/Jairam/Training/Transcripts/Audio - 27_01.txt
Detected encoding: utf-8
Audio file path: /home/ubuntu/Jairam/Training/Audio/Audio - 3_30.wav
Transcript file path: /home/ubuntu/Jairam/Training/Transcripts/Audio - 3_30.txt
Detected encoding: utf-8
Audio file path: /home/ubuntu/Jairam/Training/Audio/Audio - 14_24.wav
Transcript file path: /home/ubuntu/Jairam/Training/Transcripts/Audio - 14_24.txt
Detected encoding: utf-8
Audio file path: /home/ubuntu/Jairam/Training/Audio/Audio - 13_16.wav
Transcript file path: /home/ubuntu/Jairam/Training/Transcripts/Audio - 13_16.txt
Detected encoding: utf-8
Audio file path: /home/ubuntu/Jairam/Training/Audio/Audio - 5_11.wav
Transcript file path: /home/ubuntu/Jairam/Training/Transcripts/Audio - 5_11.txt
Detected encoding: utf-8
Audio file path: /home/ubuntu/Jairam/Training/Audio/Audio - 25_04.wav
Transcript file path: /home/ubuntu/Jairam/Training/Tra

In [3]:
df

,path,transcript
0,/home/ubuntu/Jairam/Training/Audio/Audio - 27_...,என் பெயர் ராபியா.\nசாப்டியா இந்தப் பணியை செய்வ...
1,/home/ubuntu/Jairam/Training/Audio/Audio - 3_3...,என்னென்ன இருக்குது அரிசி எண்ணெய் பருப்பு சீனி ...
2,/home/ubuntu/Jairam/Training/Audio/Audio - 14_...,பழைய நகையை இன்னைக்கு ரேட்டுக்கு எடுத்துக் எடுத...
3,/home/ubuntu/Jairam/Training/Audio/Audio - 13_...,அந்த அந்த டிக்கெட் எவ்ளோ அதுக்கு இடையில போம்போ...
4,/home/ubuntu/Jairam/Training/Audio/Audio - 5_1...,நான் கான்டக்ட்ல வரணும் இந்த ஸ்டேட் பேங்க் எங்க...
...,...,...
903,/home/ubuntu/Jairam/Training/Audio/Audio - 33_...,சுறா மீன் ருக்கா என்ன வெல\nஎன்ன கூடுதலா கொரவா\...
904,/home/ubuntu/Jairam/Training/Audio/Audio - 29_...,ஓ...அண்ணதானம் வந்து எல்லாருக்கும் சாப்டுமா அண்...
905,/home/ubuntu/Jairam/Training/Audio/Audio - 16_...,"டீ,காபி குடிக்கபோது உசுப்புவாங்களா எழுப்பமாட்..."
906,/home/ubuntu/Jairam/Training/Audio/Audio - 35_...,நம்ம எல்லாரும் பொள்ளாச்சிக்கு போகணும் நாலு நாள...


In [4]:
df['path'][1]

'/home/ubuntu/Jairam/Training/Audio/Audio - 3_30.wav'

In [21]:
# creating training and testing csv files
from sklearn.model_selection import train_test_split

df = pd.read_csv("dataset.csv")

# Split the DataFrame into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Save the train and test DataFrames to separate CSV files
train_df.to_csv("train_data.csv", index=False)
test_df.to_csv("test_data.csv", index=False)

# Print the shapes of the train and test DataFrames
print("Train data shape:", train_df.shape)
print("Test data shape:", test_df.shape)

Train data shape: (726, 4)
Test data shape: (182, 4)


## Whisper Model

## Pre-Processing

In [1]:
from datasets import Dataset
from datasets import Audio
import gc
import pandas as pd

# loading the csv files
train_df = pd.read_csv("train_data.csv")
test_df = pd.read_csv("test_data.csv")

print(len(test_df))
train_df

In [3]:
# creating the dataset using the class methods Dataset.from_pandas()
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_dataset[0]

{'transcript': 'சீட்டு இல்லாம குடுப்பீங்களா இல்ல டாக்டர் சீட்டு கண்டிப்பா வேணுமா அந்த மாத்திரைக்குரிய எஸ்பியிரி டேட் எப்போ முடியுது அந்தமாதிரி பாத்து குடுங்க பா கொஞ்சம் நா வந்து கொஞ்சம் லேட்டா ஆச்சுனாக இதையுரும்ல அதுக்காக பாத்து ',
 'audio': '/home/ubuntu/Jairam/Training/Audio/Audio - 10_31.wav'}

In [4]:
# create arrays of each audio file and append those values as a column in the above datasets.
# To do this we will use the cast_column function from Dataset.
train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=16000))
test_dataset = test_dataset.cast_column("audio", Audio(sampling_rate=16000))

### Prepare Feature Extractor, Tokenizer and Data
### Load WhisperFeatureExtractor and Tokenizer

In [5]:
# !pip install transformers

In [6]:
# !pip install --upgrade datasets transformers accelerate soundfile librosa evaluate jiwer tensorboard gradio

In [5]:
## import feature extractor and tokenizer
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer

feature_extractor = WhisperFeatureExtractor.from_pretrained("vasista22/whisper-tamil-medium")
tokenizer = WhisperTokenizer.from_pretrained("vasista22/whisper-tamil-medium",task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
## Combine To Create A WhisperProcessor

from transformers import WhisperProcessor
processor = WhisperProcessor.from_pretrained("vasista22/whisper-tamil-medium", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Preparing data for the model
    * generating log-mel spectrogram
    * creating label_ids using tokenizer

In [9]:
train_dataset[0]

{'transcript': 'சீட்டு இல்லாம குடுப்பீங்களா இல்ல டாக்டர் சீட்டு கண்டிப்பா வேணுமா அந்த மாத்திரைக்குரிய எஸ்பியிரி டேட் எப்போ முடியுது அந்தமாதிரி பாத்து குடுங்க பா கொஞ்சம் நா வந்து கொஞ்சம் லேட்டா ஆச்சுனாக இதையுரும்ல அதுக்காக பாத்து ',
 'audio': {'path': '/home/ubuntu/Jairam/Training/Audio/Audio - 10_31.wav',
  'array': array([ 0.0019205 ,  0.00218401,  0.00186531, ..., -1.16827393,
         -0.98321271, -0.48510537]),
  'sampling_rate': 16000}}

In [7]:
input_str = train_dataset[0]["transcript"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")


Input:                 சீட்டு இல்லாம குடுப்பீங்களா இல்ல டாக்டர் சீட்டு கண்டிப்பா வேணுமா அந்த மாத்திரைக்குரிய எஸ்பியிரி டேட் எப்போ முடியுது அந்தமாதிரி பாத்து குடுங்க பா கொஞ்சம் நா வந்து கொஞ்சம் லேட்டா ஆச்சுனாக இதையுரும்ல அதுக்காக பாத்து 
Decoded w/ special:    <|startoftranscript|><|transcribe|><|notimestamps|>சீட்டு இல்லாம குடுப்பீங்களா இல்ல டாக்டர் சீட்டு கண்டிப்பா வேணுமா அந்த மாத்திரைக்குரிய எஸ்பியிரி டேட் எப்போ முடியுது அந்தமாதிரி பாத்து குடுங்க பா கொஞ்சம் நா வந்து கொஞ்சம் லேட்டா ஆச்சுனாக இதையுரும்ல அதுக்காக பாத்து <|endoftext|>
Decoded w/out special: சீட்டு இல்லாம குடுப்பீங்களா இல்ல டாக்டர் சீட்டு கண்டிப்பா வேணுமா அந்த மாத்திரைக்குரிய எஸ்பியிரி டேட் எப்போ முடியுது அந்தமாதிரி பாத்து குடுங்க பா கொஞ்சம் நா வந்து கொஞ்சம் லேட்டா ஆச்சுனாக இதையுரும்ல அதுக்காக பாத்து 
Are equal:             True


In [8]:
def prepare_dataset(examples):
    # compute log-Mel input features from input audio array
    audio = examples["audio"]
    examples["input_features"] = feature_extractor(
        audio["array"], sampling_rate=16000).input_features[0]

    del examples["audio"]

    sentences = examples["transcript"]
    
    # encode target text to label ids
    examples["labels"] = tokenizer(sentences).input_ids

    del examples["transcript"]
    return examples

#### Using the dataset’s.map method, we can apply the data preparation function to each of our training examples

In [9]:
train_dataset = train_dataset.map(prepare_dataset, num_proc=2)
test_dataset = test_dataset.map(prepare_dataset, num_proc=2)

Map (num_proc=2):   0%|          | 0/726 [00:00<?, ? examples/s]

/home/ubuntu/environments/speech2text/lib/python3.10/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/ubuntu/environments/speech2text/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


Map (num_proc=2):   0%|          | 0/182 [00:00<?, ? examples/s]

### Training and Evaluation
    * Defining a data-collator to handle input data and label spearetly
    * The word error rate (WER) metric will be used to measure the effectiveness of the model. A compute metrics function must be created to handle this calculation.
    * Loading a pre-trained checkpoint: A pre-trained checkpoint must be loaded and properly set up for training.
    * Defining the training arguments: The trainer will use these to create the training schedule.

In [10]:
train_dataset

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 726
})

In [10]:
## data-collator
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [11]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [12]:
## Evalution metric
import evaluate
metric = evaluate.load("wer")

In [13]:
# evaluation metric function
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id
    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    
    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    
    return {"wer": wer}


In [14]:
# Loading a Pre-Trained Checkpoint
from transformers import WhisperForConditionalGeneration
model = WhisperForConditionalGeneration.from_pretrained("vasista22/whisper-tamil-medium")

In [15]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [17]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./pretrainedwhisper-medium-native-v3",  # change to a repo name of your choice
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=200,
    max_steps=2000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
    optim="adamw_bnb_8bit"
)

In [18]:
## training
from transformers import Seq2SeqTrainer
# early_stopping_callback = EarlyStoppingCallback(early_stopping_threshold=0.001, early_stopping_patience=2)
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    # callbacks=[early_stopping_callback]
)

In [ ]:
trainer.train()

/home/ubuntu/environments/speech2text/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
500,0.239100,0.583140,73.051948
1000,0.021000,0.759181,71.672078
1500,0.002900,0.826172,71.327110
2000,0.000800,0.859534,71.367695


/home/ubuntu/environments/speech2text/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/ubuntu/environments/speech2text/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/ubuntu/environments/speech2tex

TrainOutput(global_step=2000, training_loss=0.14357925122743473, metrics={'train_runtime': 11816.4536, 'train_samples_per_second': 0.677, 'train_steps_per_second': 0.169, 'total_flos': 8.1444271620096e+18, 'train_loss': 0.14357925122743473, 'epoch': 10.99})

In [25]:
model.save_pretrained(training_args.output_dir)
processor.save_pretrained(training_args.output_dir)